# Analyze Simulation Results
This tutorial will use the {py:class}`~pyfibers.fiber.Fiber` from the [fiber creation tutorial](1_create_fiber.ipynb) and simulation results from the [simulation and threshold search](2_sim_and_activation.ipynb) tutorials. We will analyze the response in transmembrane electric potential (Vm) and gating variables to extracellular stimulation of a fiber over space and time.

## Create the fiber and set up simulation
As before, we create fiber, waveform, potentials and {py:class}`~pyfibers.stimulation.ScaledStim` object.

In [ ]:
from pyfibers import build_fiber, FiberModel, ScaledStim
import numpy as np
from scipy.interpolate import interp1d

# create fiber model
n_nodes = 25
fiber = build_fiber(FiberModel.MRG_INTERPOLATION, diameter=10, n_nodes=n_nodes)
print(fiber)

# Setup for simulation. Add zeros at the beginning so we get some baseline for visualization
time_step = 0.001
time_stop = 20
start, on, off = 0, 0.1, 0.2  # milliseconds
waveform = interp1d(
    [start, on, off, time_stop], [0, 1, 0, 0], kind="previous"
)  # monophasic rectangular pulse

fiber.potentials = fiber.point_source_potentials(0, 250, fiber.length / 2, 1, 10)

# Create stimulation object
stimulation = ScaledStim(waveform=waveform, dt=time_step, tstop=time_stop)

## Run Simulation

As before, we can simulate the response to a single stimulation pulse.

In [ ]:
stimamp = -1.5  # mA
ap, time = stimulation.run_sim(stimamp, fiber)
print(f'Number of action potentials detected: {ap}')
print(f'Time of last action potential detection: {time} ms')

Before running the simulation, we did not tell the fiber to save any data. Therefore, no transmembrane potential (Vm) or gating variable information was stored. We can confirm this using Python's ``hasattr()`` command.

In [ ]:
# checks if the fiber object has the given attribute:
# transmembrane_potentials (vm), gating variables (gating) and transmembrane currents (im)
saved_vm = fiber.vm is not None
print(f"Saved Vm?\n\t{saved_vm}\n")

saved_gating = fiber.gating is not None
print(f"Saved gating?\n\t{saved_gating}")

saved_im = fiber.im is not None
print(f"Saved Im?\n\t{saved_im}")

Let's control the fiber to save the membrane voltage and gating variables and then re-run the simulation. Note that you can record from specific sections of the fiber, record at specific timepoints, or record at a given time step (larger than the simulation time step). For more info, see the {py:class}`Fiber API Documentation <pyfibers.fiber.Fiber>`. Here, we will proceed with the default usage, which records for all nodes (rather than at every section) at every simulation time step.

In [ ]:
fiber.record_vm()  # save membrane voltage
fiber.record_gating()  # save gating variables
fiber.record_im()  # save membrane current
ap, time = stimulation.run_sim(-1.5, fiber)

Now that we have saved membrane voltage and gating variables, let's take a look at them.

In [ ]:
print(fiber.vm)
print(fiber.gating)
print(fiber.im)

We have a neuron {py:class}`Vector <h.Vector>` object for each node of the fiber.

NOTE: By default MRG fibers are created with passive end nodes (see that the first and last values are "None") to prevent initiation of action potentials at the terminals due to edge-effects. We are simulating the response of a fiber of finite length local to the site of stimulation.

Next, let's plot the transmembrane voltage for one end compartment and the center compartment to visualize the fiber response to stimulation.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font_scale=1.5, style='whitegrid', palette='colorblind')

end_node = 1  # not zero since it was passive and therefore has no data to show!
center_node = fiber.loc_index(0.5)

plt.figure()
plt.plot(
    np.array(stimulation.time),
    list(fiber.vm[end_node]),
    label='end node',
    color='royalblue',
    linewidth=2,
)
plt.plot(
    np.array(stimulation.time),
    list(fiber.vm[center_node]),
    label='center node',
    color='mediumturquoise',
    linewidth=2,
)
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('$V_m$ $(mV)$')
ax2 = plt.gca().twinx()
ax2.plot(
    np.array(stimulation.time),
    stimamp * waveform(stimulation.time),
    'k--',
    label='Stimulus',
)
ax2.legend(loc=4)
ax2.grid(False)
plt.xlim([0, 2])
plt.ylabel('Stimulation amplitude (mA)')
plt.show()

We can plot a heatmap of the voltage across all compartments over time.

In [ ]:
import pandas as pd

data = pd.DataFrame(np.array(fiber.vm[1:-1]))
vrest = fiber[0].e_pas
print('Membrane rest voltage:', vrest)
g = sns.heatmap(
    data,
    cbar_kws={'label': '$V_m$ $(mV)$'},
    cmap='seismic',
    vmax=np.amax(data.values) + vrest,
    vmin=-np.amax(data.values) + vrest,
)
plt.xlim([0, 2000])
plt.ylabel('Node index')
plt.xlabel('Time (ms)')
tick_locs = np.linspace(0, len(np.array(stimulation.time)[:2000]), 9)
labels = [round(np.array(stimulation.time)[int(ind)], 2) for ind in tick_locs]
g.set_xticks(ticks=tick_locs, labels=labels)
plt.title(
    'Membrane voltage over time\
          \nRed=depolarized, Blue=hyperpolarized'
)

Running a threshold search will also save our variables. Let's try plotting Vm at threshold.

In [ ]:
amp, ap = stimulation.find_threshold(fiber)
print(f'Activation threshold: {amp} mA')

In [ ]:
# plot vm
plt.figure()
plt.plot(
    np.array(stimulation.time),
    list(fiber.vm[end_node]),
    label='end node',
    color='royalblue',
    linewidth=2,
)
plt.plot(
    np.array(stimulation.time),
    list(fiber.vm[center_node]),
    label='center node',
    color='mediumturquoise',
    linewidth=2,
)
plt.xlim([0, 2])
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('$V_m$ $(mV)$')
ax2 = plt.gca().twinx()
ax2.plot(
    np.array(stimulation.time),
    amp * waveform(stimulation.time),
    'k--',
    label='Stimulus',
)
ax2.legend(loc=4)
ax2.grid(False)
plt.ylabel('Stimulation amplitude (mA)')
plt.show()

In [ ]:
# plot heatmap
data = pd.DataFrame(np.array(fiber.vm[1:-1]))
vrest = fiber[0].e_pas
print('Membrane rest voltage:', vrest)
g = sns.heatmap(
    data,
    cbar_kws={'label': '$V_m$ $(mV)$'},
    cmap='seismic',
    vmax=np.amax(data.values) + vrest,
    vmin=-np.amax(data.values) + vrest,
)
plt.xlim([0, 2000])
tick_locs = np.linspace(0, len(np.array(stimulation.time)[:2000]), 9)
labels = [round(np.array(stimulation.time)[int(ind)], 2) for ind in tick_locs]
g.set_xticks(ticks=tick_locs, labels=labels)
plt.ylabel('Node index')
plt.xlabel('Time (ms)')
plt.title(
    'Membrane voltage over time. \
          \nRed=depolarized, Blue=hyperpolarized'
)

Let's take a look at the gating variables.

In [ ]:
# plot gating variables
plt.figure()
for var in fiber.gating:
    plt.plot(np.array(stimulation.time), list(fiber.gating[var][6]), label=var)
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Gating probability')
ax2 = plt.gca().twinx()
ax2.plot(
    np.array(stimulation.time),
    amp * waveform(stimulation.time),
    'k--',
    label='Stimulus',
)
ax2.legend(loc=4)
ax2.grid(False)
plt.xlim([0, 2])
plt.ylabel('Stimulation amplitude (mA)')
plt.show()

...and the transmembrane currents. 

In [ ]:
plt.figure()
fig, axs = plt.subplots(3, 1, figsize=(5, 5), sharex=True, gridspec_kw={'hspace': 0.3})
plt.sca(axs[0])
# plot stimulus
plt.plot(
    np.array(stimulation.time),
    amp * waveform(stimulation.time),
    'k--',
    label='Stimulus',
)
plt.title('Stimulus')
plt.sca(axs[1])
# plot membrane voltage
plt.plot(
    np.array(stimulation.time),
    list(fiber.vm[center_node]),
    color='mediumturquoise',
    linewidth=2,
    label='$V_m$',
)
# plot im
plt.plot(
    np.array(stimulation.time),
    list(fiber.im[center_node]),
    color='mediumturquoise',
    linewidth=2,
    label='$I_m$',
    ls='--',
)
plt.title('Center node')
plt.legend()
plt.sca(axs[2])
# plot end node
plt.plot(
    np.array(stimulation.time),
    list(fiber.vm[end_node]),
    color='royalblue',
    linewidth=2,
    label='$V_m$',
)
plt.plot(
    np.array(stimulation.time),
    list(fiber.im[end_node]),
    color='royalblue',
    linewidth=2,
    label='$I_m$',
    ls='--',
)
plt.title('End node')
plt.legend()
plt.xlim([0, 2])
axs[2].set_xlabel('Time (ms)')
plt.show()

Finally, we can use the data to make videos, which can help visualize how the fiber variables change over time.

In [ ]:
from matplotlib.animation import FuncAnimation

# Parameters
skip = 10  # Process every 10th timestep
stop_time = 2  # Stop after 2 milliseconds

# Calculate total number of timesteps available
total_steps = len(fiber.vm[0])  # Assuming each node in fiber.vm is a list or array
n_frames = int(stop_time / (time_step * skip))

ylim = (np.amin(list(fiber.vm[1:-1])), np.amax(list(fiber.vm[1:-1])))

# Set up the x-axis (node indices or positions)
node_indices = range(1, len(fiber.vm) - 1)  # Adjust to match your data

# Set up the figure and axis
fig, ax = plt.subplots()
ax.set_ylim(ylim)
ax.set_xlim(min(node_indices), max(node_indices))
ax.set_xlabel('Node index')
ax.set_ylabel('$V_m$')

(line,) = ax.plot([], [], lw=3, color='mediumturquoise')
title = ax.set_title('')


# Initialize the frame
def init():  # noqa: D103
    line.set_data([], [])
    title.set_text('')
    return line, title


# Update function for animation
def update(frame):  # noqa: D103
    ind = frame * skip
    if ind >= total_steps:  # Safety check
        return line, title
    y_data = [v[ind] for v in fiber.vm[1:-1]]
    line.set_data(node_indices, y_data)
    title.set_text(f'Time: {ind * time_step:.1f} ms')
    return line, title


# Create animation
ani = FuncAnimation(
    fig, update, frames=n_frames, init_func=init, blit=True, interval=20
)

# Adjust layout to prevent clipping
plt.tight_layout()

# Display the animation
from IPython.display import HTML

plt.close(fig)  # Close the static plot
HTML(ani.to_jshtml())

You can use the same technique to plot videos of other data, such as transmembrane current or gating variables!